In [ ]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.19.0
2.0.2
4.56.1
4.0.0


In [ ]:
import pandas as pd
from datasets import Dataset

def parse_rating_file(file_path):
    """한국어 영화 리뷰 데이터 파일을 안전하게 파싱하는 함수"""
    data = {
        'id': [],
        'document': [],
        'label': []
    }

    with open(file_path, 'r', encoding='utf-8') as f:
        # 헤더 스킵
        next(f)

        for line_num, line in enumerate(f, 1):
            try:
                line = line.strip()
                if not line:  # 빈 줄 스킵
                    continue

                parts = line.split('\t')
                if len(parts) >= 3:
                    # ID는 정수로 변환
                    review_id = int(parts[0])
                    # 문서 텍스트 (탭이 포함된 경우를 대비해 나머지 모든 부분을 합침)
                    document = '\t'.join(parts[1:-1]) if len(parts) > 3 else parts[1]
                    # 라벨은 마지막 컬럼
                    label = int(parts[-1])

                    # 빈 문서나 잘못된 라벨 체크
                    if not document.strip():
                        print(f"Line {line_num}: Empty document, skipping")
                        continue
                    if label not in [0, 1]:
                        print(f"Line {line_num}: Invalid label {label}, skipping")
                        continue

                    data['id'].append(review_id)
                    data['document'].append(document.strip())
                    data['label'].append(label)
                else:
                    print(f"Line {line_num}: Invalid format (expected 3 columns), skipping")
            except ValueError as e:
                print(f"Line {line_num}: Value conversion error {e}, skipping")
            except Exception as e:
                print(f"Line {line_num}: Unexpected error {e}, skipping")

    return pd.DataFrame(data)

In [ ]:
def load_rating_datasets(train_path, test_path):
    """훈련 및 테스트 데이터를 로드하고 기본 통계를 출력하는 함수"""
    # 데이터 로드
    train_df = parse_rating_file(train_path)
    test_df = parse_rating_file(test_path)

    # 기본 통계 출력
    print("=== 훈련 데이터 정보 ===")
    print(f"총 샘플 수: {len(train_df)}")
    print(f"라벨 분포:")
    print(train_df['label'].value_counts().sort_index())
    print(f"평균 문서 길이: {train_df['document'].str.len().mean():.1f}자")
    print(f"최대 문서 길이: {train_df['document'].str.len().max()}자")

    print("\n=== 테스트 데이터 정보 ===")
    print(f"총 샘플 수: {len(test_df)}")
    print(f"라벨 분포:")
    print(test_df['label'].value_counts().sort_index())
    print(f"평균 문서 길이: {test_df['document'].str.len().mean():.1f}자")
    print(f"최대 문서 길이: {test_df['document'].str.len().max()}자")

    # 샘플 데이터 출력
    print("\n=== 훈련 데이터 샘플 ===")
    for i in range(min(3, len(train_df))):
        row = train_df.iloc[i]
        print(f"ID: {row['id']}")
        print(f"라벨: {row['label']} ({'긍정' if row['label'] == 1 else '부정'})")
        print(f"텍스트: {row['document']}")
        print("-" * 50)

    return train_df, test_df

In [ ]:
def convert_to_huggingface_dataset(df):
    """pandas DataFrame을 허깅페이스 Dataset으로 변환"""
    return Dataset.from_pandas(df)

In [ ]:
# 데이터 로드
train_df, test_df = load_rating_datasets('ratings_train.txt', 'ratings_test.txt')

# 허깅페이스 Dataset으로 변환
train_dataset = convert_to_huggingface_dataset(train_df)
test_dataset = convert_to_huggingface_dataset(test_df)

print(f"\n=== 허깅페이스 Dataset 변환 완료 ===")
print(f"훈련 데이터셋: {train_dataset}")
print(f"테스트 데이터셋: {test_dataset}")

Line 25858: Empty document, skipping
Line 55738: Empty document, skipping
Line 110015: Empty document, skipping
Line 126783: Empty document, skipping
Line 140722: Empty document, skipping
Line 5747: Empty document, skipping
Line 7900: Empty document, skipping
Line 27098: Empty document, skipping
=== 훈련 데이터 정보 ===
총 샘플 수: 149995
라벨 분포:
label
0    75170
1    74825
Name: count, dtype: int64
평균 문서 길이: 35.2자
최대 문서 길이: 158자

=== 테스트 데이터 정보 ===
총 샘플 수: 49997
라벨 분포:
label
0    24826
1    25171
Name: count, dtype: int64
평균 문서 길이: 35.4자
최대 문서 길이: 152자

=== 훈련 데이터 샘플 ===
ID: 9976970
라벨: 0 (부정)
텍스트: 아 더빙.. 진짜 짜증나네요 목소리
--------------------------------------------------
ID: 3819312
라벨: 1 (긍정)
텍스트: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
--------------------------------------------------
ID: 10265843
라벨: 0 (부정)
텍스트: 너무재밓었다그래서보는것을추천한다
--------------------------------------------------

=== 허깅페이스 Dataset 변환 완료 ===
훈련 데이터셋: Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 149995
})
테스트 데이터셋

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# KLUE BERT 토크나이저와 모델 로드
huggingface_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import torch

In [ ]:
# 1. 데이터 전처리 함수
def tokenize_function(examples):
    """텍스트를 토크나이징하는 함수"""
    return huggingface_tokenizer(
        examples['document'],
        truncation=True,
        padding=True,
        max_length=128,  # 메모리와 속도를 고려한 길이
        return_tensors="pt" if isinstance(examples['document'], str) else None
    )

In [ ]:
# 2. 평가 메트릭 함수
def compute_metrics(eval_pred):
    """학습 중 평가 메트릭을 계산하는 함수"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)

    return {
        'accuracy': accuracy
    }

In [ ]:
# 3. 데이터셋 준비
print("=== 데이터셋 준비 중 ===")

# DataFrame을 HuggingFace Dataset으로 변환 (이미 위에서 했다면 스킵)
if 'train_dataset' not in globals():
    train_dataset = convert_to_huggingface_dataset(train_df)
    test_dataset = convert_to_huggingface_dataset(test_df)

# 토크나이징 적용
print("토크나이징 진행 중...")
train_dataset_tokenized = train_dataset.map(tokenize_function, batched=True)
test_dataset_tokenized = test_dataset.map(tokenize_function, batched=True)

# 학습에 필요한 컬럼만 유지
train_dataset_tokenized = train_dataset_tokenized.remove_columns(['id', 'document'])
test_dataset_tokenized = test_dataset_tokenized.remove_columns(['id', 'document'])

# 라벨 컬럼명 변경 (Trainer가 'labels'를 기대함)
train_dataset_tokenized = train_dataset_tokenized.rename_column('label', 'labels')
test_dataset_tokenized = test_dataset_tokenized.rename_column('label', 'labels')

# 데이터 타입 설정
train_dataset_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(f"전처리 완료!")
print(f"훈련 데이터: {len(train_dataset_tokenized)} 샘플")
print(f"테스트 데이터: {len(test_dataset_tokenized)} 샘플")

=== 데이터셋 준비 중 ===
토크나이징 진행 중...


Map:   0%|          | 0/149995 [00:00<?, ? examples/s]

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

전처리 완료!
훈련 데이터: 149995 샘플
테스트 데이터: 49997 샘플


In [ ]:
# 4. 학습 설정
print("\n=== 학습 설정 ===")

training_args = TrainingArguments(
    output_dir='./results',          # 모델 저장 경로
    num_train_epochs=3,              # 에폭 수
    per_device_train_batch_size=16,  # 배치 크기
    per_device_eval_batch_size=16,
    warmup_steps=500,               # 워밍업 스텝
    weight_decay=0.01,              # 가중치 감소
    logging_dir='./logs',           # 로그 저장 경로
    logging_steps=100,              # 로깅 간격
    eval_strategy="steps",          # 평가 전략 (최신 버전)
    eval_steps=5000,                 # 평가 간격
    save_strategy="steps",          # 저장 전략
    save_steps=5000,                # 저장 간격
    load_best_model_at_end=True,    # 최고 모델 로드
    metric_for_best_model="accuracy", # 최고 모델 선택 기준
    greater_is_better=True,         # 높을수록 좋음
    report_to=None,                 # wandb 등 비활성화
    seed=42                         # 재현 가능성을 위한 시드
)


=== 학습 설정 ===


In [ ]:
# 5. Trainer 초기화
print("Trainer 초기화 중...")

trainer = Trainer(
    model=huggingface_model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,  # 평가용으로 테스트셋 사용
    compute_metrics=compute_metrics,
    tokenizer=huggingface_tokenizer
)

Trainer 초기화 중...


/tmp/ipython-input-2632568074.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 6. 학습 시작
print("\n=== 모델 학습 시작 ===")
print("이 과정은 시간이 걸릴 수 있습니다...")

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

# 학습 실행
train_result = trainer.train()

print("\n=== 학습 완료 ===")
print(f"최종 학습 손실: {train_result.training_loss:.4f}")


=== 모델 학습 시작 ===
이 과정은 시간이 걸릴 수 있습니다...
사용 중인 디바이스: cuda


wandb: Currently logged in as: luis-jang (luis-jang-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
5000,0.299900,0.284577,0.879453,0.879296,0.881034,0.879453
10000,0.199700,0.337629,0.892414,0.892370,0.892840,0.892414
15000,0.209300,0.345447,0.890073,0.889910,0.892923,0.890073
20000,0.166000,0.356656,0.900474,0.900472,0.900478,0.900474
25000,0.147900,0.348486,0.901494,0.901479,0.901627,0.901494



=== 학습 완료 ===
최종 학습 손실: 0.2249
